In [1]:
from logging import error
from web3.auto import w3
import json
import requests
import os
from dotenv import load_dotenv
from datetime import datetime as dt

load_dotenv()


Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


True

In [2]:
# get contract address from .env
contract_addr = os.getenv("TEST_CONTRACT_ADDRESS")
# contract_addr = os.getenv("DAN_ROOT_ADDRESS")
# contract_addr = os.getenv("GANACHE_PRIVATE_KEY")

In [3]:
contract_addr

'0xc9A18bbA5098Aaf0d88C470B391FB8ec20Ae36d2'

In [4]:
# set ABI path
abi_file = "web3TransactTestABI.json"

In [5]:
# open abi file and load as json format
with open(abi_file) as json_file:
    abi_data = json.load(json_file)

In [6]:
print(abi_data)

[{'anonymous': False, 'inputs': [{'indexed': False, 'internalType': 'uint256', 'name': 'TestEventID', 'type': 'uint256'}, {'indexed': False, 'internalType': 'string', 'name': 'testText', 'type': 'string'}], 'name': 'testEmit', 'type': 'event'}, {'anonymous': False, 'inputs': [{'indexed': False, 'internalType': 'uint256', 'name': 'TestEventID', 'type': 'uint256'}, {'indexed': False, 'internalType': 'uint256', 'name': 'createDate', 'type': 'uint256'}, {'indexed': False, 'internalType': 'string', 'name': 'URI', 'type': 'string'}], 'name': 'testEventRegistration', 'type': 'event'}, {'constant': False, 'inputs': [{'internalType': 'uint256', 'name': '_createDate', 'type': 'uint256'}, {'internalType': 'string', 'name': '_URI', 'type': 'string'}], 'name': 'registerTestEvent', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'constant': False, 'inputs': [{'internalType': 'string', 'name': '_testText

In [7]:
# init eth contract
test_contract = w3.eth.contract(address=contract_addr, abi=abi_data)

In [8]:
# Pinata IPFS functionality
pinata_headers = {
    "Content-Type": "application/json",
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY"),
}

def convertDataToJSON(content:dict):
    data = {
        "pinataOptions": {"cidVersion": 1},
        "pinataContent": content,
    }
    return json.dumps(data)

def pinJSONtoIPFS(json):
    req = requests.post(
        "https://api.pinata.cloud/pinning/pinJSONToIPFS", data=json, headers=pinata_headers
    )
    ipfs_hash = req.json()["IpfsHash"]
    return f"ipfs://{ipfs_hash}"

In [9]:
def get_testEventID_from_URI(event_URI: str):
    test_event_reg_filter = test_contract.events.testEventRegistration.createFilter(fromBlock="0x0", argument_filters={"URI": event_URI})
    test_event_registrations = test_event_reg_filter.get_all_entries()
    return test_event_registrations[0].TestEventID

def register_test_event(event_name: str, event_recipient: str, funding_goal: int, create_date):

    # convert string start and end dates to datetime (if they aren't already datetime objects)
    if not isinstance(create_date, dt):
        create_date = dt.strptime(create_date, "%Y/%m/%d")

    # convert start and end datetimes to to unix timestamps
    create_date = create_date.timestamp()

    # create charity_info dict
    test_event_info = {
        "testEventName": event_name,
        # charityEventRecipient: event_recipient,
        "goalAmount": funding_goal
    }

    # convert charity_info to json_charity_info (json format) and pin to IPFS via pinata api function
    json_event_info = convertDataToJSON(test_event_info)
    ipfs_link = pinJSONtoIPFS(json_event_info)

    tx_hash = test_contract.functions.registerTestEvent(int(create_date), ipfs_link).transact({"from": w3.eth.accounts[0]})
    receipt  = w3.eth.waitForTransactionReceipt(tx_hash)

    # test_event_id = get_testEventID_from_URI(ipfs_link)
    # charity_event_id = int(receipt['logs'][0]['data'])

    # return test_event_id
    return receipt

In [ ]:
# function registerTestEvent(uint _createDate, string memory _URI)
# event testEventRegistration(
#    uint TestEventID,
#    uint createDate,
#    string URI 
# );


In [13]:
def get_testEventID_from_URI(event_URI: str):
    test_event_reg_filter = test_contract.events.testEventRegistration.createFilter(fromBlock="0x0", argument_filters={"URI": event_URI})
    test_event_registrations = test_event_reg_filter.get_all_entries()
    return test_event_registrations[0].TestEventID

In [11]:
# test register_test_event
event_receipt = register_test_event("Dan Test Event1", "no recipient", 100, "2020/11/14")

In [12]:
event_receipt

AttributeDict({'transactionHash': HexBytes('0x4aa64fb1364b02053ea961e4fd5b34e5d7d9ac49e381fb2f88a14ae07ae5f89a'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xaadbca8f9b22b447591b1db81cab1afc7976d5ef53e3f1201b34f59a13d995ce'),
 'blockNumber': 94,
 'from': '0x13C0fAd72f393A9ea42AE90c7a868f509cF13CD6',
 'to': '0xc9A18bbA5098Aaf0d88C470B391FB8ec20Ae36d2',
 'gasUsed': 149822,
 'cumulativeGasUsed': 149822,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x4aa64fb1364b02053ea961e4fd5b34e5d7d9ac49e381fb2f88a14ae07ae5f89a'),
   'blockHash': HexBytes('0xaadbca8f9b22b447591b1db81cab1afc7976d5ef53e3f1201b34f59a13d995ce'),
   'blockNumber': 94,
   'address': '0xc9A18bbA5098Aaf0d88C470B391FB8ec20Ae36d2',
   'data': '0x0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000005faf72600000000000000000000000000000000000000000000000000000000000000060000000000000

In [22]:
# event_registration = get_testEventID_from_URI("Dan Test Event1")
# test_event_reg_filter = test_contract.events.testEventRegistration.createFilter(fromBlock="0x0", argument_filters={"URI": "Dan Test Event1"})
test_event_reg_filter = test_contract.events.testEventRegistration.createFilter(fromBlock="0x0", argument_filters={'URI': 'ipfs://bafkreicrxn3nodqm6skinatz474otlmgeoqvhad3ausgpunxryjygyri6q'})
test_event_registrations = test_event_reg_filter.get_all_entries()

In [26]:
test_event_registrations[0]

AttributeDict({'args': AttributeDict({'TestEventID': 1,
  'createDate': 1605333600,
  'URI': 'ipfs://bafkreicrxn3nodqm6skinatz474otlmgeoqvhad3ausgpunxryjygyri6q'}),
 'event': 'testEventRegistration',
 'logIndex': 0,
 'transactionIndex': 0,
 'transactionHash': HexBytes('0x4aa64fb1364b02053ea961e4fd5b34e5d7d9ac49e381fb2f88a14ae07ae5f89a'),
 'address': '0xc9A18bbA5098Aaf0d88C470B391FB8ec20Ae36d2',
 'blockHash': HexBytes('0xaadbca8f9b22b447591b1db81cab1afc7976d5ef53e3f1201b34f59a13d995ce'),
 'blockNumber': 94})

In [49]:
type(test_event_registrations[0])

web3.datastructures.AttributeDict

In [50]:
from pprint import pprint

In [51]:
from hexbytes import HexBytes

def toDict(dictToParse):
    # convert any 'AttributeDict' type found to 'dict'
    parsedDict = dict(dictToParse)
    for key, val in parsedDict.items():
        # check for nested dict structures to iterate through
        if  'dict' in str(type(val)).lower():
            parsedDict[key] = toDict(val)
        # convert 'HexBytes' type to 'str'
        elif 'HexBytes' in str(type(val)):
            parsedDict[key] = val.hex()
    return parsedDict

In [55]:
test_event_registrations_dict = toDict(test_event_registrations[0])

In [57]:
test_event_registrations_json = json.dumps(test_event_registrations_dict)


In [59]:
pprint(test_event_registrations_json)

('{"args": {"TestEventID": 1, "createDate": 1605333600, "URI": '
 '"ipfs://bafkreicrxn3nodqm6skinatz474otlmgeoqvhad3ausgpunxryjygyri6q"}, '
 '"event": "testEventRegistration", "logIndex": 0, "transactionIndex": 0, '
 '"transactionHash": '
 '"0x4aa64fb1364b02053ea961e4fd5b34e5d7d9ac49e381fb2f88a14ae07ae5f89a", '
 '"address": "0xc9A18bbA5098Aaf0d88C470B391FB8ec20Ae36d2", "blockHash": '
 '"0xaadbca8f9b22b447591b1db81cab1afc7976d5ef53e3f1201b34f59a13d995ce", '
 '"blockNumber": 94}')


In [66]:
test_event_registrations_dict['args']['TestEventID']

1